In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import torch
import numpy as np
import pandas as pd
import os
import librosa
import cv2

from pathlib import Path
from random import shuffle
from diffsynth import ModelManager, WanVideoPipeline, save_video, VideoData

In [13]:
vid_path = Path("dataset/train")
metadata = pd.read_csv("dataset/metadata.csv")
masks = Path("dataset/lip_masks")

In [14]:
# Load latents
latents_paths = [video_path for video_path in vid_path.iterdir() if video_path.suffix == ".pth"]
# shuffle(latents_paths)

In [15]:
# C, F, H, W
latent = torch.load(latents_paths[0], map_location='cpu')

In [17]:
# Load 5 latents
latents = []
audio_list = []

for latent_path in latents_paths[12:14]:
    latents.append(torch.load(latent_path)["latents"])
    name = latent_path.name.replace(".tensors.pth", "")

    caption = metadata[metadata["file_name"] == name]["text"].values[0]
    # Load audio
    audio_path = f"dataset/audio/{name.replace('.mp4', '.wav')}"

    print(os.path.exists(audio_path))

    audio_data, sr = librosa.load(audio_path, sr=None)
    
    # Add audio to list
    
    audio_list.append({
        'name': name,
        'audio': audio_data,
        'caption': caption
    })

True
True


NameError: name 'pipe' is not defined

In [18]:
latents = torch.stack(latents).to("cuda")

In [ ]:
# Play the audio
from IPython.display import Audio
display(Audio(audio_list[1]['audio'], rate=41000))

In [ ]:
pipe.vae.to(pipe.device)

In [30]:
out = pipe.vae.decode(latents, device='cuda')

In [ ]:
for i in range(out.shape[0]):
    video = out[i].permute(1, 2, 3, 0).cpu().float()
    save_video(video, f"video_{i}.mp4", fps=30)